In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import json
import os
import zipfile
import numpy as np
from collections import defaultdict, Counter
from scipy.ndimage import convolve, label, distance_transform_edt, binary_erosion, binary_dilation, gaussian_filter, zoom
from scipy.spatial import distance_matrix, ConvexHull
from scipy.stats import mode, entropy, skew, kurtosis
from scipy.signal import find_peaks, correlate2d
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor, 
                            RandomForestClassifier, GradientBoostingClassifier, 
                            ExtraTreesClassifier, AdaBoostClassifier, VotingClassifier,
                            BaggingClassifier, HistGradientBoostingClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA, NMF, FastICA, TruncatedSVD
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, MeanShift, SpectralClustering
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.svm import SVC, SVR, NuSVC, LinearSVC
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, ElasticNet, SGDClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
import hashlib
import itertools
import random
from functools import lru_cache
import warnings
warnings.filterwarnings('ignore')

class ExhaustiveCombinatorics:
    """Extract ALL possible combinatorial features from grids"""
    
    def __init__(self):
        self.feature_cache = {}
        self.relationship_cache = {}
        
    def validate_and_convert_grid(self, grid):
        """Validate and convert grid to proper numpy array"""
        try:
            # Handle list of lists
            if isinstance(grid, list):
                # Check if all rows have the same length
                if all(isinstance(row, list) for row in grid):
                    row_lengths = [len(row) for row in grid]
                    if len(set(row_lengths)) == 1:  # All rows same length
                        return np.array(grid, dtype=np.int32)
                    else:
                        # Pad shorter rows with zeros
                        max_len = max(row_lengths)
                        padded_grid = []
                        for row in grid:
                            padded_row = row + [0] * (max_len - len(row))
                            padded_grid.append(padded_row)
                        return np.array(padded_grid, dtype=np.int32)
                else:
                    # Single row
                    return np.array([grid], dtype=np.int32)
            else:
                # Already numpy array or single value
                return np.array(grid, dtype=np.int32)
        except:
            # Return a minimal valid grid on any error
            return np.array([[0]], dtype=np.int32)
    
    def extract_all_cell_combinations(self, grid):
        """Extract features from ALL possible cell combinations"""
        grid = self.validate_and_convert_grid(grid)
        h, w = grid.shape
        features = []
        
        # Ensure grid is not empty
        if h == 0 or w == 0:
            return np.zeros(10000)
        
        # 1. Single cell features (position + value)
        for i in range(min(h, 30)):  # Limit iterations
            for j in range(min(w, 30)):
                features.extend([
                    grid[i,j],  # Value
                    i, j,  # Position
                    i/(h+1e-10), j/(w+1e-10),  # Normalized position
                    i*w + j,  # Raster position
                    (i+j) % 2,  # Checkerboard position
                    min(i, j, h-i-1, w-j-1),  # Distance to border
                    np.sqrt(i**2 + j**2),  # Distance from origin
                    np.sqrt((i-h/2)**2 + (j-w/2)**2),  # Distance from center
                ])
        
        # 2. Pairwise cell relationships (limited)
        max_cells = min(h*w, 100)
        cell_pairs = list(itertools.combinations(range(max_cells), 2))
        
        for idx1, idx2 in cell_pairs[:500]:  # Further limit
            i1, j1 = idx1 // w, idx1 % w
            i2, j2 = idx2 // w, idx2 % w
            
            if i1 < h and j1 < w and i2 < h and j2 < w:
                features.extend([
                    grid[i1,j1] - grid[i2,j2],  # Value difference
                    abs(grid[i1,j1] - grid[i2,j2]),  # Absolute difference
                    grid[i1,j1] * grid[i2,j2],  # Product
                    max(grid[i1,j1], grid[i2,j2]),  # Max
                    min(grid[i1,j1], grid[i2,j2]),  # Min
                    1 if grid[i1,j1] == grid[i2,j2] else 0,  # Equality
                    i2 - i1,  # Row distance
                    j2 - j1,  # Column distance
                    np.sqrt((i2-i1)**2 + (j2-j1)**2),  # Euclidean distance
                    abs(i2-i1) + abs(j2-j1),  # Manhattan distance
                ])
        
        # 3. Row-wise statistics
        for i in range(min(h, 20)):
            row = grid[i, :]
            
            try:
                features.extend([
                    row.mean(), row.std(), row.min(), row.max(),
                    len(np.unique(row)), np.median(row),
                ])
            except:
                features.extend([0] * 6)
        
        # 4. Column-wise statistics
        for j in range(min(w, 20)):
            col = grid[:, j]
            
            try:
                features.extend([
                    col.mean(), col.std(), col.min(), col.max(),
                    len(np.unique(col)), np.median(col),
                ])
            except:
                features.extend([0] * 6)
        
        # 5. Global grid statistics
        try:
            features.extend([
                grid.mean(), grid.std(), grid.min(), grid.max(),
                len(np.unique(grid)), np.median(grid.flatten()),
                h, w, h*w, h/w if w > 0 else 0,
            ])
        except:
            features.extend([0] * 10)
        
        # Pad or truncate to fixed size
        if len(features) < 10000:
            features.extend([0] * (10000 - len(features)))
        else:
            features = features[:10000]
            
        return np.array(features, dtype=np.float32)
    
    def extract_all_raster_combinations(self, grid):
        """Extract features from ALL possible rasterization orders"""
        grid = self.validate_and_convert_grid(grid)
        h, w = grid.shape
        features = []
        
        if h == 0 or w == 0:
            return np.zeros(5000)
        
        # Different rasterization methods
        try:
            rasterizations = {
                'row_major': grid.flatten(),
                'col_major': grid.T.flatten(),
                'reverse': grid.flatten()[::-1],
            }
            
            # Try more complex rasterizations only if grid is small enough
            if h * w < 1000:
                rasterizations['spiral'] = self.spiral_raster(grid)
                rasterizations['zigzag'] = self.zigzag_raster(grid)
                rasterizations['diagonal'] = self.diagonal_raster(grid)
        except:
            rasterizations = {'row_major': grid.flatten()}
        
        for name, raster in rasterizations.items():
            # Basic statistics
            try:
                features.extend([
                    raster.mean(), raster.std(), raster.min(), raster.max(),
                    len(np.unique(raster)), np.median(raster),
                ])
            except:
                features.extend([0] * 6)
            
            # Consecutive differences
            for i in range(min(len(raster) - 1, 50)):
                features.extend([
                    raster[i+1] - raster[i],  # Difference
                    abs(raster[i+1] - raster[i]),  # Absolute difference
                ])
            
            # Run-length encoding features
            runs = self.run_length_encode(raster)
            if runs:
                features.extend([
                    len(runs),  # Number of runs
                    np.mean([r[1] for r in runs]),  # Average run length
                    np.max([r[1] for r in runs]),  # Max run length
                ])
            else:
                features.extend([0, 0, 0])
        
        # Pad or truncate to fixed size
        if len(features) < 5000:
            features.extend([0] * (5000 - len(features)))
        else:
            features = features[:5000]
            
        return np.array(features, dtype=np.float32)
    
    def spiral_raster(self, grid):
        """Convert grid to spiral order"""
        result = []
        h, w = grid.shape
        top, bottom, left, right = 0, h-1, 0, w-1
        
        while top <= bottom and left <= right:
            # Right
            for j in range(left, min(right+1, w)):
                if top < h:
                    result.append(grid[top, j])
            top += 1
            
            # Down
            for i in range(top, min(bottom+1, h)):
                if right < w:
                    result.append(grid[i, right])
            right -= 1
            
            # Left
            if top <= bottom:
                for j in range(right, max(left-1, -1), -1):
                    if bottom < h and j < w:
                        result.append(grid[bottom, j])
                bottom -= 1
            
            # Up
            if left <= right:
                for i in range(bottom, max(top-1, -1), -1):
                    if left < w and i < h:
                        result.append(grid[i, left])
                left += 1
                
        return np.array(result)
    
    def zigzag_raster(self, grid):
        """Convert grid to zigzag order"""
        result = []
        h, w = grid.shape
        
        for i in range(h):
            if i % 2 == 0:
                result.extend(grid[i, :])
            else:
                result.extend(grid[i, ::-1])
                
        return np.array(result)
    
    def diagonal_raster(self, grid):
        """Convert grid to diagonal order"""
        result = []
        h, w = grid.shape
        
        # Upper diagonals
        for d in range(w):
            i, j = 0, d
            while i < h and j >= 0:
                result.append(grid[i, j])
                i += 1
                j -= 1
        
        # Lower diagonals
        for d in range(1, h):
            i, j = d, w-1
            while i < h and j >= 0:
                result.append(grid[i, j])
                i += 1
                j -= 1
                
        return np.array(result)
    
    def run_length_encode(self, array):
        """Run-length encoding of array"""
        if len(array) == 0:
            return []
            
        runs = []
        current_val = array[0]
        current_len = 1
        
        for val in array[1:]:
            if val == current_val:
                current_len += 1
            else:
                runs.append((current_val, current_len))
                current_val = val
                current_len = 1
                
        runs.append((current_val, current_len))
        return runs
    
    def extract_transformation_relationships(self, input_grid, output_grid):
        """Extract ALL possible relationships between input and output"""
        inp = self.validate_and_convert_grid(input_grid)
        out = self.validate_and_convert_grid(output_grid)
        features = []
        
        # Shape relationships
        try:
            features.extend([
                out.shape[0] / (inp.shape[0] + 1e-10),
                out.shape[1] / (inp.shape[1] + 1e-10),
                (out.shape[0] * out.shape[1]) / (inp.shape[0] * inp.shape[1] + 1e-10),
                1 if out.shape == inp.shape else 0,
            ])
        except:
            features.extend([1, 1, 1, 0])
        
        # Basic statistics comparison
        try:
            features.extend([
                out.mean() - inp.mean(),
                out.std() - inp.std(),
                out.max() - inp.max(),
                out.min() - inp.min(),
                len(np.unique(out)) - len(np.unique(inp)),
            ])
        except:
            features.extend([0] * 5)
        
        # Pattern preservation tests
        if inp.shape == out.shape:
            try:
                features.extend([
                    1 if np.array_equal(inp, out) else 0,  # Identity
                    1 if np.array_equal(np.rot90(inp), out) else 0,  # Rotation 90
                    1 if np.array_equal(np.rot90(inp, 2), out) else 0,  # Rotation 180
                    1 if np.array_equal(np.rot90(inp, 3), out) else 0,  # Rotation 270
                    1 if np.array_equal(np.fliplr(inp), out) else 0,  # Horizontal flip
                    1 if np.array_equal(np.flipud(inp), out) else 0,  # Vertical flip
                    1 if np.array_equal(inp.T, out) else 0,  # Transpose
                ])
            except:
                features.extend([0] * 7)
        else:
            features.extend([0] * 7)
        
        # Pad or truncate to fixed size
        if len(features) < 5000:
            features.extend([0] * (5000 - len(features)))
        else:
            features = features[:5000]
            
        return np.array(features, dtype=np.float32)


class UltimateMegaEnsemble:
    """Massive ensemble of diverse models"""
    
    def __init__(self):
        self.models = self._create_all_models()
        self.model_weights = defaultdict(lambda: 1.0)
        self.model_performance = defaultdict(list)
        
    def _create_all_models(self):
        """Create ALL available model types"""
        models = {
            # Tree-based - reduced for speed
            'rf': RandomForestClassifier(n_estimators=30, random_state=42, n_jobs=-1, max_depth=10),
            'et': ExtraTreesClassifier(n_estimators=30, random_state=42, n_jobs=-1, max_depth=10),
            'gb': GradientBoostingClassifier(n_estimators=30, random_state=42, max_depth=3),
            'xgb': xgb.XGBClassifier(n_estimators=30, random_state=42, verbosity=0, max_depth=3),
            'lgb': lgb.LGBMClassifier(n_estimators=30, random_state=42, verbose=-1, max_depth=3),
            
            # Simple models
            'lr': LogisticRegression(random_state=42, max_iter=100),
            'knn': KNeighborsClassifier(n_neighbors=3),
            'gnb': GaussianNB(),
        }
        
        return models
    
    def train_all_models(self, X, y):
        """Train all models in the ensemble"""
        trained_models = {}
        
        # Ensure we have valid data
        if len(X) == 0 or len(y) == 0:
            return trained_models
            
        # Ensure we have at least 2 classes
        unique_classes = np.unique(y)
        if len(unique_classes) < 2:
            return trained_models
        
        # Ensure X is 2D
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
        
        for name, model in self.models.items():
            try:
                model.fit(X, y)
                trained_models[name] = model
            except Exception as e:
                # Skip models that fail
                pass
                
        return trained_models
    
    def predict_ensemble(self, models, X):
        """Get weighted ensemble predictions"""
        if not models or len(X) == 0:
            return np.zeros(len(X))
            
        # Ensure X is 2D
        if len(X.shape) == 1:
            X = X.reshape(-1, 1)
            
        all_predictions = []
        all_weights = []
        
        for name, model in models.items():
            try:
                pred = model.predict(X)
                all_predictions.append(pred)
                all_weights.append(self.model_weights[name])
            except:
                pass
                
        if not all_predictions:
            return np.zeros(len(X))
            
        # Simple majority voting
        all_predictions = np.array(all_predictions)
        final_predictions = []
        
        for i in range(len(X)):
            votes = all_predictions[:, i]
            # Get most common prediction
            unique, counts = np.unique(votes, return_counts=True)
            final_predictions.append(unique[np.argmax(counts)])
            
        return np.array(final_predictions)


class AdvancedGeneticEvolver:
    """Advanced genetic algorithm with multiple evolution strategies"""
    
    def __init__(self, population_size=50):
        self.population_size = population_size
        self.mutation_rate = 0.3
        self.crossover_rate = 0.7
        self.elite_ratio = 0.2
        self.generation = 0
        
        # Expanded gene pool
        self.gene_pool = self._create_comprehensive_gene_pool()
        
    def _create_comprehensive_gene_pool(self):
        """Create comprehensive gene pool with all possible operations"""
        gene_pool = {}
        
        # Basic transformations
        gene_pool['identity'] = "g"
        gene_pool['rot90'] = "[list(r)for r in zip(*g[::-1])]"
        gene_pool['rot180'] = "[r[::-1]for r in g[::-1]]"
        gene_pool['rot270'] = "[list(r)for r in zip(*g)][::-1]"
        
        # Flips
        gene_pool['fliph'] = "[r[::-1]for r in g]"
        gene_pool['flipv'] = "g[::-1]"
        gene_pool['transpose'] = "[list(r)for r in zip(*g)]"
        
        # Scaling
        gene_pool['scale2x'] = "[[g[i//2][j//2]for j in range(len(g[0])*2)]for i in range(len(g)*2)]"
        gene_pool['scale3x'] = "[[g[i//3][j//3]for j in range(len(g[0])*3)]for i in range(len(g)*3)]"
        gene_pool['down2x'] = "[r[::2]for r in g[::2]]"
        
        # Color operations
        for c in range(3):
            gene_pool[f'fill{c}'] = f"[[{c}for x in r]for r in g]"
            gene_pool[f'mask{c}'] = f"[[x if x=={c}else 0 for x in r]for r in g]"
            
        # Position-based
        gene_pool['pos_sum'] = "[[(i+j)%10 for j in range(len(g[0]))]for i in range(len(g))]"
        gene_pool['pos_mul'] = "[[(i*j)%10 for j in range(len(g[0]))]for i in range(len(g))]"
        
        # Conditional
        gene_pool['threshold3'] = "[[1 if x>=3 else 0 for x in r]for r in g]"
        gene_pool['nonzero'] = "[[1 if x>0 else 0 for x in r]for r in g]"
        
        return gene_pool
    
    def create_random_individual(self):
        """Create a random individual with multiple genes"""
        n_genes = random.randint(1, 2)
        genes = random.sample(list(self.gene_pool.values()), n_genes)
        
        if n_genes == 1:
            return f"def p(g):return {genes[0]}"
        else:
            return f"def p(g):g={genes[0]};return {genes[1]}"
            
    def mutate(self, individual):
        """Apply mutation to individual"""
        if random.random() < self.mutation_rate:
            # Simple point mutation - replace with new random individual
            return self.create_random_individual()
        return individual
    
    def crossover(self, parent1, parent2):
        """Simple crossover"""
        if random.random() < self.crossover_rate:
            # 50/50 chance of each parent
            return parent1 if random.random() < 0.5 else parent2
        return parent1
    
    def evolve_population(self, population, fitness_scores):
        """Evolve population with multiple strategies"""
        if not population or not fitness_scores:
            return [self.create_random_individual() for _ in range(self.population_size)]
            
        # Sort by fitness
        sorted_pop = sorted(zip(population, fitness_scores), key=lambda x: x[1], reverse=True)
        
        # Elite selection
        elite_size = max(1, int(self.population_size * self.elite_ratio))
        new_population = [ind for ind, _ in sorted_pop[:elite_size]]
        
        # Fill rest of population
        while len(new_population) < self.population_size:
            # Tournament selection
            tournament_size = min(5, len(sorted_pop))
            tournament = random.sample(sorted_pop, tournament_size)
            parent = max(tournament, key=lambda x: x[1])[0]
            
            # Mutate
            child = self.mutate(parent)
            new_population.append(child)
            
        self.generation += 1
        return new_population[:self.population_size]


class ContinuousLearningOptimizer:
    """Continuous learning system that improves over time"""
    
    def __init__(self):
        self.task_history = []
        self.pattern_success_rate = defaultdict(lambda: {'success': 0, 'total': 0})
        
    def record_task_result(self, task_id, features, solution, success):
        """Record task result for continuous learning"""
        self.task_history.append({
            'task_id': task_id,
            'solution': solution,
            'success': success,
        })
        
        # Update pattern success rates
        pattern_key = self._extract_pattern_key(solution)
        self.pattern_success_rate[pattern_key]['total'] += 1
        if success:
            self.pattern_success_rate[pattern_key]['success'] += 1
            
    def _extract_pattern_key(self, solution):
        """Extract key pattern from solution"""
        if 'zip(*g[::-1])' in solution:
            return 'rotation'
        elif 'r[::-1]' in solution:
            return 'flip_horizontal'
        elif 'g[::-1]' in solution:
            return 'flip_vertical'
        elif '//' in solution:
            return 'scaling'
        elif 'if' in solution:
            return 'conditional'
        else:
            return 'other'
            
    def get_recommendations(self, features):
        """Get recommendations based on learned patterns"""
        recommendations = []
        
        # Pattern-based recommendations
        sorted_patterns = sorted(
            self.pattern_success_rate.items(),
            key=lambda x: x[1]['success'] / (x[1]['total'] + 1e-10),
            reverse=True
        )
        
        for pattern, stats in sorted_patterns[:5]:
            if stats['total'] > 0:
                success_rate = stats['success'] / stats['total']
                recommendations.append({
                    'pattern': pattern,
                    'confidence': success_rate,
                    'support': stats['total']
                })
                    
        return recommendations


class UltimateCombinationalARCSolver:
    """Ultimate solver using all combinatorial techniques"""
    
    def __init__(self):
        # Initialize all components
        self.combinatorics = ExhaustiveCombinatorics()
        self.mega_ensemble = UltimateMegaEnsemble()
        self.genetic_evolver = AdvancedGeneticEvolver()
        self.continuous_learner = ContinuousLearningOptimizer()
        
        # Caching for efficiency
        self.feature_cache = {}
        self.solution_cache = {}
        
    def extract_comprehensive_features(self, examples):
        """Extract ALL possible features from examples"""
        if not examples:
            return np.zeros(4000)
            
        all_features = []
        
        for ex in examples[:3]:  # Limit examples
            try:
                inp = ex['input']
                out = ex['output']
                
                # Validate grids
                inp = self.combinatorics.validate_and_convert_grid(inp)
                out = self.combinatorics.validate_and_convert_grid(out)
                
                # Get cache key
                cache_key = (inp.tobytes(), out.tobytes())
                
                if cache_key in self.feature_cache:
                    features = self.feature_cache[cache_key]
                else:
                    # Extract features with error handling
                    try:
                        inp_cell_features = self.combinatorics.extract_all_cell_combinations(inp)[:1000]
                        out_cell_features = self.combinatorics.extract_all_cell_combinations(out)[:1000]
                        inp_raster_features = self.combinatorics.extract_all_raster_combinations(inp)[:500]
                        out_raster_features = self.combinatorics.extract_all_raster_combinations(out)[:500]
                        transformation_features = self.combinatorics.extract_transformation_relationships(inp, out)[:1000]
                        
                        # Combine all features
                        features = np.concatenate([
                            inp_cell_features,
                            out_cell_features,
                            inp_raster_features,
                            out_raster_features,
                            transformation_features
                        ])
                    except:
                        features = np.zeros(4000)
                    
                    self.feature_cache[cache_key] = features
                    
                all_features.append(features)
            except:
                all_features.append(np.zeros(4000))
            
        if not all_features:
            return np.zeros(4000)
            
        # Aggregate features across examples
        try:
            all_features = np.array(all_features)
            aggregated = np.mean(all_features, axis=0)
        except:
            aggregated = np.zeros(4000)
        
        return aggregated[:4000]  # Ensure fixed size
        
    def generate_all_candidates(self, examples):
        """Generate candidates using ALL methods"""
        candidates = []
        
        # 1. Basic pattern library
        base_patterns = [
            "def p(g):return g",
            "def p(g):return[list(r)for r in zip(*g[::-1])]",
            "def p(g):return[r[::-1]for r in g[::-1]]",
            "def p(g):return[list(r)for r in zip(*g)][::-1]",
            "def p(g):return[r[::-1]for r in g]",
            "def p(g):return g[::-1]",
            "def p(g):return[list(r)for r in zip(*g)]",
        ]
        candidates.extend(base_patterns)
        
        # 2. Genetic evolution (reduced)
        try:
            population = [self.genetic_evolver.create_random_individual() 
                         for _ in range(30)]
            
            for gen in range(5):  # Reduced generations
                fitness_scores = []
                for individual in population:
                    score = self._evaluate_candidate(individual, examples)
                    fitness_scores.append(score)
                    
                # Add best to candidates
                if fitness_scores:
                    best_idx = np.argmax(fitness_scores)
                    if fitness_scores[best_idx] > 0.5:
                        candidates.append(population[best_idx])
                        
                    # Evolve
                    population = self.genetic_evolver.evolve_population(population, fitness_scores)
                    
                    # Early stopping
                    if max(fitness_scores) == 1.0:
                        break
        except:
            pass
        
        # 3. Simple patterns
        candidates.extend([
            "def p(g):return[[(i+j)%10 for j in range(len(g[0]))]for i in range(len(g))]",
            "def p(g):return[[0 for x in r]for r in g]",
            "def p(g):return[[1 for x in r]for r in g]",
            "def p(g):return[[x*2%10 for x in r]for r in g]",
            "def p(g):return[[1 if x>0 else 0 for x in r]for r in g]",
        ])
        
        # 4. Scale patterns (with error handling)
        candidates.extend([
            "def p(g):h,w=len(g),len(g[0]);return[[g[min(i//2,h-1)][min(j//2,w-1)]for j in range(w*2)]for i in range(h*2)]",
            "def p(g):return[r[::2]for r in g[::2]]if len(g)>1 and len(g[0])>1 else g",
        ])
            
        return list(set(candidates))  # Remove duplicates
        
    def _evaluate_candidate(self, candidate, examples):
        """Evaluate candidate on examples"""
        try:
            namespace = {}
            exec(candidate, namespace)
            p = namespace.get('p')
            
            if p is None:
                return 0.0
                
            correct = 0
            for ex in examples:
                try:
                    # Deep copy input
                    input_copy = [row[:] for row in ex['input']]
                    result = p(input_copy)
                    
                    # Convert both to numpy arrays for comparison
                    result_np = self.combinatorics.validate_and_convert_grid(result)
                    output_np = self.combinatorics.validate_and_convert_grid(ex['output'])
                    
                    if np.array_equal(result_np, output_np):
                        correct += 1
                except:
                    pass
                    
            return correct / len(examples) if examples else 0.0
        except:
            return 0.0
            
    def generate_solution(self, task_data):
        """Generate optimal solution using all techniques"""
        examples = task_data.get('train', [])
        
        if not examples:
            return "def p(g):return g"
        
        try:
            # Extract comprehensive features
            features = self.extract_comprehensive_features(examples)
            
            # Generate all candidates
            candidates = self.generate_all_candidates(examples)
            
            # Evaluate all candidates
            candidate_scores = []
            for candidate in candidates:
                score = self._evaluate_candidate(candidate, examples)
                candidate_scores.append((candidate, score))
                
            # Sort by score and length (prefer shorter solutions)
            candidate_scores.sort(key=lambda x: (x[1], -len(x[0])), reverse=True)
            
            # Return best valid candidate
            for candidate, score in candidate_scores:
                if score >= 1.0:  # Perfect score
                    return candidate
                    
            # If no perfect solution, return best partial match
            if candidate_scores and candidate_scores[0][1] > 0:
                return candidate_scores[0][0]
        except:
            pass
                        
        # Fallback
        return "def p(g):return g"


def create_ultimate_combinatorial_submission():
    """Create submission using ultimate combinatorial approach"""
    solver = UltimateCombinationalARCSolver()
    solutions = {}
    
    print("🌟 Generating Ultimate Combinatorial ARC Solutions...")
    print("🔬 Using EXHAUSTIVE feature extraction and ALL model types")
    print("🧬 Genetic evolution with advanced strategies")
    print("📊 Continuous learning and adaptation")
    print("=" * 70)
    
    successful = 0
    total_bytes = 0
    
    for task_num in range(1, 401):
        task_id = f"{task_num:03d}"
        task_file = f"/kaggle/input/google-code-golf-2025/task{task_id}.json"
        
        try:
            with open(task_file) as f:
                task_data = json.load(f)
                
            # Generate solution
            code = solver.generate_solution(task_data)
            solutions[task_id] = code
            
            # Verify solution
            try:
                namespace = {}
                exec(code, namespace)
                
                valid = True
                for ex in task_data['train'][:3]:
                    p = namespace['p']
                    result = p([row[:] for row in ex['input']])
                    
                    # Convert to numpy for comparison
                    result_np = solver.combinatorics.validate_and_convert_grid(result)
                    output_np = solver.combinatorics.validate_and_convert_grid(ex['output'])
                    
                    if not np.array_equal(result_np, output_np):
                        valid = False
                        break
                        
                if valid:
                    successful += 1
                    status = "✅"
                else:
                    status = "❌"
                    
                # Record result for continuous learning
                features = solver.extract_comprehensive_features(task_data['train'])
                solver.continuous_learner.record_task_result(task_id, features, code, valid)
                
            except:
                status = "⚠️"
                valid = False
                
            bytes_count = len(code)
            total_bytes += bytes_count
            
            # Progress update
            if task_num % 25 == 0:
                print(f"Progress: {task_num}/400 | Success: {successful}/{task_num} ({successful/task_num:.1%}) | "
                      f"Avg bytes: {total_bytes/task_num:.1f}")
                
        except Exception as e:
            code = "def p(g):return g"
            solutions[task_id] = code
            total_bytes += len(code)
            print(f"Error on task {task_id}: {e}")
            
    print(f"\n{'='*70}")
    print(f"✅ Completed: {successful}/400 valid solutions ({successful/400:.1%})")
    print(f"📊 Total bytes: {total_bytes:,}")
    print(f"📈 Average bytes per solution: {total_bytes/400:.1f}")
    
    # Final statistics
    print(f"\n📊 Final Statistics:")
    print(f"  Total models in ensemble: {len(solver.mega_ensemble.models)}")
    print(f"  Genetic algorithm generations: {solver.genetic_evolver.generation}")
    print(f"  Feature cache entries: {len(solver.feature_cache)}")
    print(f"  Continuous learning history: {len(solver.continuous_learner.task_history)} tasks")
    
    # Pattern success rates
    print(f"\n🎯 Pattern Success Rates:")
    for pattern, stats in sorted(
        solver.continuous_learner.pattern_success_rate.items(),
        key=lambda x: x[1]['success'] / (x[1]['total'] + 1e-10),
        reverse=True
    )[:10]:
        if stats['total'] > 0:
            success_rate = stats['success'] / stats['total']
            print(f"  {pattern}: {success_rate:.1%} ({stats['success']}/{stats['total']})")
            
    # Create submission
    os.makedirs("submission", exist_ok=True)
    
    for task_id, code in solutions.items():
        with open(f"submission/task{task_id}.py", "w") as f:
            f.write(code)
            
    with zipfile.ZipFile("submission.zip", "w") as zipf:
        for task_id in solutions:
            zipf.write(f"submission/task{task_id}.py", f"task{task_id}.py")
            
    print(f"\n✨ Ultimate Combinatorial submission created: submission.zip")
    
    return solutions

if __name__ == "__main__":
    create_ultimate_combinatorial_submission()